In [5]:
# -*- coding: utf-8 -*-
# -*- author : Vincent Roduit -*-
# -*- date : 2023-11-12 -*-
# -*- Last revision: 2023-11-14 -*-
# -*- python version : 3.12.0 -*-
# -*- Description: Create the dataset of all users -*-

In [6]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
import powerlaw
from statsmodels.stats import diagnostic
from datetime import datetime
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%reload_ext autoreload

In [8]:
#import functions
from read.read_functions import *
from read.pickle_functions import *
from create_all_users import create_all_users
from create_rating_statistic import create_ratings_stat

# Download and save datas
### Data paths

In [9]:
beer_advocate_path = '../datas/BeerAdvocate/'
matched_beer_data_path = '../datas/matched_beer_data/'
rate_beer_path = '../datas/RateBeer/'

advocate_beers_path = beer_advocate_path + 'beers.csv'
advovate_breweries_path = beer_advocate_path + 'breweries.csv'
advocate_ratings_path = beer_advocate_path + 'ratings.txt'
advocate_reviews_path = beer_advocate_path + 'reviews.txt'
advocate_users_path = beer_advocate_path + 'users.csv'

matched_beer_beers = matched_beer_data_path + 'beers.csv'
matched_beer_breweries = matched_beer_data_path + 'breweries.csv'
matched_beer_ratings_ba = matched_beer_data_path + 'ratings_ba.txt'
matched_beer_ratings_rb = matched_beer_data_path + 'ratings_rb.txt'
matched_beer_ratings = matched_beer_data_path + 'ratings.csv'
matched_beer_users_approx = matched_beer_data_path + 'users_approx.csv'
matched_beer_users = matched_beer_data_path + 'users.csv'

rate_beer_beers = rate_beer_path + 'beers.csv'
rate_beer_breweries = rate_beer_path + 'breweries.csv'
rate_beer_users = rate_beer_path + 'users.csv'
rate_beer_ratings = rate_beer_path + 'ratings.txt'
rate_beer_reviews = rate_beer_path + 'reviews.txt'

contries_path = '../datas/countries/countries.csv'


In [10]:
beer_advocate_path_pickle = '../datas/BeerAdvocate/pickles/df_advocate_'
matched_beer_data_path_pickle = '../datas/matched_beer_data/pickles/df_matched_beer_'
rate_beer_path_pickle = '../datas/RateBeer/pickles/df_rate_beer_'

advocate_beers_path_pickle = beer_advocate_path_pickle + 'beers.pkl'
advovate_breweries_path_pickle = beer_advocate_path_pickle + 'breweries.pkl'
advocate_ratings_path_pickle = beer_advocate_path_pickle + 'ratings.pkl'
advocate_reviews_path_pickle = beer_advocate_path_pickle + 'reviews.pkl'
advocate_users_path_pickle = beer_advocate_path_pickle + 'users.pkl'

matched_beer_beers_pickle = matched_beer_data_path_pickle + 'beers.pkl'
matched_beer_breweries_pickle = matched_beer_data_path_pickle + 'breweries.pkl'
matched_beer_ratings_ba_pickle = matched_beer_data_path_pickle + 'ratings_ba.pkl'
matched_beer_ratings_rb_pickle = matched_beer_data_path_pickle + 'ratings_rb.pkl'
matched_beer_ratings_pickle = matched_beer_data_path_pickle + 'ratings.pkl'
matched_beer_users_approx_pickle = matched_beer_data_path_pickle + 'users_approx.pkl'
matched_beer_users_pickle = matched_beer_data_path_pickle + 'users.csv'

rate_beer_beers_pickle = rate_beer_path_pickle + 'beers.pkl'
rate_beer_breweries_pickle = rate_beer_path_pickle + 'breweries.pkl'
rate_beer_users_pickle = rate_beer_path_pickle + 'users.pkl'
rate_beer_ratings_pickle = rate_beer_path_pickle + 'ratings.pkl'
rate_beer_reviews_pickle = rate_beer_path_pickle + 'reviews.pkl'

In [11]:
#Define list of datas for each website
datas_matched_beer_names = ['df_matched_beer_beers','df_matched_beer_breweries','df_matched_beer_ratings_ba','df_matched_beer_ratings_rb','df_matched_beer_ratings','df_matched_beer_users_approx','df_matched_beer_users']
datas_advocate_names = ['df_advocate_beers','df_advocate_breweries','df_advocate_ratings','df_advocate_reviews','df_advocate_users']
datas_rate_beer_names = ['df_rate_beer_beers','df_rate_beer_breweries','df_rate_beer_users','df_rate_beer_ratings','df_rate_beer_reviews']

### import datas
$\color{Red}{\text{Attention}}$ : Run the celluls in this section only if the datasets stored as pickle are not available.

### Import datas stored as pickle

In [12]:
#import raw data sets
df_matched_beer_beers, df_matched_beer_breweries, df_matched_beer_ratings_ba, df_matched_beer_ratings_rb, df_matched_beer_ratings, df_matched_beer_users_approx, df_matched_beer_users = load_datas('matched_beer_data', datas_matched_beer_names)
df_advocate_beers, df_advocate_breweries, df_advocate_ratings, df_advocate_reviews, df_advocate_users = load_datas('BeerAdvocate', datas_advocate_names)
df_rate_beer_beers,df_rate_beer_breweries,df_rate_beer_users,df_rate_beer_ratings,df_rate_beer_reviews = load_datas('RateBeer', datas_rate_beer_names)

Loading df_matched_beer_beers...
Loading df_matched_beer_breweries...
Loading df_matched_beer_ratings_ba...
Loading df_matched_beer_ratings_rb...
Loading df_matched_beer_ratings...
Loading df_matched_beer_users_approx...
Loading df_matched_beer_users...
Loading df_advocate_beers...
Loading df_advocate_breweries...
Loading df_advocate_ratings...
Loading df_advocate_reviews...
Loading df_advocate_users...
Loading df_rate_beer_beers...
Loading df_rate_beer_breweries...
Loading df_rate_beer_users...
Loading df_rate_beer_ratings...
Loading df_rate_beer_reviews...


## Store dataFrame into picke format
Use this section to store the datasets in pickle (normally done once)

In [ ]:
datas_rate_beer = [df_rate_beer_beers,df_rate_beer_breweries,df_rate_beer_users,df_rate_beer_ratings,df_rate_beer_reviews]
datas_matched_beer = [df_matched_beer_beers,df_matched_beer_breweries,df_matched_beer_ratings_ba,df_matched_beer_ratings_rb,df_matched_beer_ratings,df_matched_beer_users_approx,df_matched_beer_users]
datas_advocate_beer = [df_advocate_beers,df_advocate_breweries,df_advocate_ratings,df_advocate_reviews,df_advocate_users]

In [ ]:
df_rate_beer_ratings = read_txt(rate_beer_ratings)

In [ ]:
save_pickle(df_rate_beer_ratings, '../datas/RateBeer/pickles/df_rate_beer_ratings.pkl')

In [ ]:
#Saving datas
save_datas('RateBeer', datas_rate_beer,datas_rate_beer_names)
save_datas('matched_beer_data', datas_matched_beer,datas_matched_beer_names)
save_datas('BeerAdvocate', datas_advocate_beer,datas_advocate_names)

# Initial data analysis

## 0. Preprocessing datasets

In [70]:
from cleaning_functions.matched_beer import *
from cleaning_functions.rate_beer import *
from cleaning_functions.advocate import *

In [71]:
mb_users = deepcopy(df_matched_beer_users)
users, mb_users_index = clean_mb_users(mb_users)
advocate_users = deepcopy(df_advocate_users)
advocate_users = clean_advocate_users(advocate_users)
rb_users = deepcopy(df_rate_beer_users)
rb_users = clean_rb_users(rb_users)

In [104]:
df_adv_beers = deepcopy(df_advocate_beers)

In [105]:
df_adv_beers = clean_adocate_beers(df_adv_beers)

In [106]:
df_rb_beers = deepcopy(df_rate_beer_beers)
df_rb_beers = clean_rb_beers(df_rb_beers)

In [107]:
rb_ratings = deepcopy(df_rate_beer_ratings)
rb_ratings = clean_rb_ratings(rb_ratings)

KeyboardInterrupt: 

In [108]:
mb_beers = deepcopy(df_matched_beer_beers)

In [109]:
mb_beers = clean_mb_beers(mb_beers)

In [110]:
from create_all_beers import*

In [112]:
test = create_all_beers(df_adv_beers,df_rb_beers,mb_beers)

In [113]:
test.head()

,ba_brewery_id,rb_brewery_id,beer_unique_id,brewery,style,beer_name
0,ba_39912,NaN,ba_166064,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,Nashe Moskovskoe
1,ba_39912,NaN,ba_166065,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,Nashe Pivovskoe
2,ba_39912,NaN,ba_166066,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,Nashe Shakhterskoe
3,ba_39912,NaN,ba_166067,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,Nashe Zhigulevskoe
4,ba_39912,NaN,ba_166063,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,Zhivoe


In [115]:
test.shape[0] == df_adv_beers.shape[0] + df_rb_beers.shape[0] - mb_beers.shape[0]

True

In [ ]:
advocate_ratings = deepcopy(df_advocate_ratings)
advocate_ratings = clean_advocate_ratings(advocate_ratings)

In [ ]:
df_adv_beers = deepcopy(df_advocate_beers)
df_adv_beers = clean_adocate_beers(df_adv_beers)

In [ ]:
df_rb_beers = deepcopy(df_rate_beer_beers)
df_rb_beers = clean_rb_beers(df_rb_beers)

In [ ]:
df_adv_beers = df_adv_beers[['beer_id','beer_name','brewery_id','brewery_name','style']]
df_rb_beers = df_rb_beers[['beer_id','beer_name','brewery_id','brewery_name','style']]

## Merging Datasets

In [ ]:
df_all_users = create_all_users(advocate_users,mb_users,rb_users)

In [ ]:
df_ratings_stat = create_ratings_stat(advocate_ratings,rb_ratings, df_all_users)

In [ ]:
df_advocate_ratings_parameters = deepcopy(df_advocate_ratings[['appearance','aroma','palate','taste','overall','rating']])
df_advocate_ratings_parameters = df_advocate_ratings_parameters.astype(float)
df_rate_beer_reviews_parameters = deepcopy(df_rate_beer_reviews[['appearance','aroma','palate','taste','overall','rating']])
df_rate_beer_reviews_parameters = df_rate_beer_reviews_parameters.astype(float)

In [ ]:
advocate_appearance = np.sort(df_advocate_ratings_parameters['appearance'].astype(float).unique())
advocate_aroma = np.sort(df_advocate_ratings_parameters['aroma'].astype(float).unique())
advocate_palate = np.sort(df_advocate_ratings_parameters['palate'].astype(float).unique())
advocate_taste = np.sort(df_advocate_ratings_parameters['taste'].astype(float).unique())
advocate_overall = np.sort(df_advocate_ratings_parameters['overall'].astype(float).unique())
rate_beer_appearance = np.sort(df_rate_beer_reviews_parameters['appearance'].astype(float).unique())
rate_beer_aroma = np.sort(df_rate_beer_reviews_parameters['aroma'].astype(float).unique())
rate_beer_palate = np.sort(df_rate_beer_reviews_parameters['palate'].astype(float).unique())
rate_beer_taste = np.sort(df_rate_beer_reviews_parameters['taste'].astype(float).unique())
rate_beer_overall = np.sort(df_rate_beer_reviews_parameters['overall'].astype(float).unique())

# Make sure all arrays are of the same length
max_length = max(len(advocate_appearance), len(advocate_aroma), len(advocate_palate), len(advocate_taste), len(advocate_overall), len(rate_beer_appearance), len(rate_beer_aroma), len(rate_beer_palate), len(rate_beer_taste), len(rate_beer_overall))

advocate_appearance = np.pad(advocate_appearance, (0, max_length - len(advocate_appearance)), mode='constant')
advocate_aroma = np.pad(advocate_aroma, (0, max_length - len(advocate_aroma)), mode='constant')
advocate_palate = np.pad(advocate_palate, (0, max_length - len(advocate_palate)), mode='constant')
advocate_taste = np.pad(advocate_taste, (0, max_length - len(advocate_taste)), mode='constant')
advocate_overall = np.pad(advocate_overall, (0, max_length - len(advocate_overall)), mode='constant')
rate_beer_appearance = np.pad(rate_beer_appearance, (0, max_length - len(rate_beer_appearance)), mode='constant')
rate_beer_aroma = np.pad(rate_beer_aroma, (0, max_length - len(rate_beer_aroma)), mode='constant')
rate_beer_palate = np.pad(rate_beer_palate, (0, max_length - len(rate_beer_palate)), mode='constant')
rate_beer_taste = np.pad(rate_beer_taste, (0, max_length - len(rate_beer_taste)), mode='constant')
rate_beer_overall = np.pad(rate_beer_overall, (0, max_length - len(rate_beer_overall)), mode='constant')

data = {'advocate_appearance': advocate_appearance,
    'rate_beer_appearance': rate_beer_appearance,
    'advocate_aroma': advocate_aroma,
    'rate_beer_aroma': rate_beer_aroma,
    'advocate_palate': advocate_palate,
    'rate_beer_palate': rate_beer_palate,
    'advocate_taste': advocate_taste,
    'rate_beer_taste': rate_beer_taste,
    'advocate_overall': advocate_overall,
    'rate_beer_overall': rate_beer_overall}

df = pd.DataFrame(data)

In [ ]:
advocate_appearance_floor = np.round(df_advocate_ratings_parameters['appearance'].astype(float))
advocate_appearance_floor.dropna(inplace = True)
advocate_appearance = df_advocate_ratings_parameters['appearance'].astype(float)
advocate_appearance.dropna(inplace = True)

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
ttest_ind(advocate_appearance_floor, advocate_appearance)

In [ ]:
max_rating_advocate = max(df_advocate_ratings_parameters.rating)
min_rating_advocate = min(df_advocate_ratings_parameters.rating)
print('Max rating advocate: ', max_rating_advocate)
print('Min rating advocate: ', min_rating_advocate)

In [ ]:
max_rating_rate_beer = max(df_rate_beer_reviews_parameters.rating)
min_rating_rate_beer = min(df_rate_beer_reviews_parameters.rating)
print('Max rating rate beer: ', max_rating_rate_beer)
print('Min rating rate beer: ', min_rating_rate_beer)

In [ ]:
df

## Analysis of the distribution
 1. Beer Advocate

In [ ]:
Advocate_users_per_country = deepcopy(pd.DataFrame(df_advocate_users.groupby('location')['user_id'].count()))
Advocate_users_per_country.columns = ['nb_users']
Advocate_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)
Advocate_users_per_country[Advocate_users_per_country.index.str.startswith('United States')].agg('sum')

Advocate_users_per_country.loc['United States', 'nb_users'] = Advocate_users_per_country[
    Advocate_users_per_country.index.str.startswith('United')
]['nb_users'].sum()
Advocate_users_per_country.drop(labels=Advocate_users_per_country[Advocate_users_per_country.index.str.startswith('United States,')].index, inplace=True)
Advocate_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)    

In [ ]:
sns.barplot(x=Advocate_users_per_country.index[:10], y=Advocate_users_per_country['nb_users'][:10])
plt.title('Number of users per country for BeerAdvocate')
plt.ylabel('Number of users')
plt.semilogy()
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.hist(data=df_advocate_users, x='nbr_ratings', bins=100000, cumulative=-1, histtype='step')
plt.title('Cumulative histogram of the number of ratings per user for BeerAdvocate')
plt.xlabel('Number of users')
plt.ylabel('Number of ratings')
plt.loglog()
plt.show()

In [ ]:
# Define the power law function
def power_law(x, alpha):
    return x**(-alpha)

In [ ]:
fit = powerlaw.Fit(df_advocate_users['nbr_ratings'].values)

print('alpha =', fit.alpha)

# R: loglikelihood ratio of the two distributions if positive the first distribution is better
R, p = fit.distribution_compare('power_law', 'exponential', normalized_ratio=True)
print('R =', R)
print('p =', p)

powerlaw.plot_pdf(df_advocate_users['nbr_ratings'].values, color='b')
fit.power_law.plot_pdf(color='r', linestyle='--')

plt.title('Power law distribution of the number of ratings per user for Beer Advocate')
plt.xlabel('Number of ratings')
plt.ylabel('Probability density')
plt.show()

2. Rate Beer

In [ ]:
rate_beer_users_per_country = deepcopy(pd.DataFrame(df_rate_beer_users.groupby('location')['user_id'].count()))
rate_beer_users_per_country.columns = ['nb_users']
rate_beer_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)

rate_beer_users_per_country.loc['United States', 'nb_users'] = rate_beer_users_per_country[
    rate_beer_users_per_country.index.str.startswith('United')
]['nb_users'].sum()
rate_beer_users_per_country.drop(labels=rate_beer_users_per_country[rate_beer_users_per_country.index.str.startswith('United States,')].index, inplace=True)
rate_beer_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)   

In [ ]:
sns.barplot(x=rate_beer_users_per_country.index[:10], y=rate_beer_users_per_country['nb_users'][:10])
plt.title('Number of users per country for Matched Beer')
plt.ylabel('Number of users')
plt.semilogy()
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.hist(data=df_rate_beer_users, x='nbr_ratings', bins=100000, cumulative=-1, histtype='step')
plt.title('Cumulative histogram of the number of ratings per user for Rate Beer')
plt.xlabel('Number of users')
plt.ylabel('Number of ratings')
plt.loglog()
plt.show()

In [ ]:
fit = powerlaw.Fit(df_rate_beer_users['nbr_ratings'].values)

print('alpha =', fit.alpha)

# R: loglikelihood ratio of the two distributions if positive the first distribution is better
R, p = fit.distribution_compare('power_law', 'exponential', normalized_ratio=True)
print('R =', R)
print('p =', p)

powerlaw.plot_pdf(df_rate_beer_users['nbr_ratings'].values, color='b')
fit.power_law.plot_pdf(color='r', linestyle='--')

plt.title('Power law distribution of the number of ratings per user for RateBeer')
plt.xlabel('Number of ratings')
plt.ylabel('Probability density')
plt.show()